In [ ]:
import numpy as np
import itertools
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from itertools import combinations

# Load the dataset
data = pd.read_csv('filtered_data.csv')

# Get the column names
all_columns = ['COM-1', 'COM-2', 'POP-1', 'STA-1', 'STA-2', 'STA-3', 'STA-4', 'STA-5', 'STA-6', 'STA-7', 'STA-8', 'STA-9', 'TEC-1', 'TEC-2', 'TEC-3', 'TEC-4']

# Dictionary to store scores for each combination
scores_dict = {}

# Number of iterations
num_iterations = 100

# Run the process multiple times
for _ in range(num_iterations):
    for x_columns_size in range(1, len(all_columns) + 1):
        for x_columns_combination in combinations(all_columns, x_columns_size):
            # Check if the combination size is 1
            if len(x_columns_combination) == 1:
                # Extract features (X) and target (y)
                X_features = data[list(x_columns_combination)]
                for y_column in all_columns:
                    if y_column not in x_columns_combination:
                        y_target = data[y_column]

                        # Split the data into training and testing sets
                        X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2)

                        # Perform linear regression
                        model = LinearRegression()
                        model.fit(X_train, y_train)

                        # Calculate the score
                        score = model.score(X_test, y_test)

                        # Store the score in the dictionary
                        key = f"X:{x_columns_combination[0]}, y:{y_column}"
                        if key not in scores_dict:
                            scores_dict[key] = []
                        scores_dict[key].append(score)

# Print the average scores for each combination
for key, scores in scores_dict.items():
    average_score = sum(scores) / len(scores)
    print(f"{key}, Average Score: {average_score}")


In [ ]:
#plotting the Scatter plot to check relationship between Sal and Temp
sns.lmplot(x ="Horizontal", y ="Vertical", data = df_binary, order = 2, ci = None)
plt.grid()
plt.show()


In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and fit the logistic regression model with increased max_iter
regr = LogisticRegression(max_iter=1000)
regr.fit(X_train_scaled, y_train.ravel())

# Make predictions on the scaled test set
y_pred = regr.predict(X_test_scaled)

# Evaluate the model
accuracy = regr.score(X_test_scaled, y_test)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.neural_network import MLPClassifier

# Create and fit the neural network model
mlp_classifier = MLPClassifier()
mlp_classifier.fit(X_train, y_train.ravel())

# Evaluate the model
print(mlp_classifier.score(X_test, y_test))


In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming X is your feature matrix
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Choose a range of values for k
k_values = range(2, 20)

# Calculate the silhouette score for each value of k
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Plot the silhouette scores
plt.plot(k_values, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs. Number of Clusters')
plt.show()

# Choose the optimal k (e.g., from the elbow method)
optimal_k = 3

# Apply k-means clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to the original dataset
df_clustered = df.copy()
df_clustered['Cluster'] = clusters

# Explore characteristics of each cluster
for cluster_id in range(optimal_k):
    cluster_data = df_clustered[df_clustered['Cluster'] == cluster_id]
    print(f'\nCluster {cluster_id}:\n')
    print(cluster_data.describe())


In [ ]:
import autosklearn.regression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming X and y are your feature and target variables
# X, y = ...

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the AutoML regressor
automl = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=120, per_run_time_limit=30, n_jobs=-1)
automl.fit(X_train, y_train)

# Make predictions on the test set
y_pred = automl.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Access additional information about the AutoML process
print(automl.sprint_statistics())


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load your dataset into a DataFrame (replace 'data.csv' with your actual dataset file)
df = pd.read_csv('filtered_data.csv')


# Calculate the percentage decrease for each indicator
df['Percentage_Decrease'] = ((df['STA-2'] - df['STA-2'].shift(1)) / df['STA-2'].shift(1)) * 100

# Define a binary target variable based on 20% decrease threshold
df['Sustainability'] = df['Percentage_Decrease'].apply(lambda x: 'unsustainable' if x >= -20 else 'sustainable')

# Drop the first row as it doesn't have a previous value for percentage calculation
df = df.dropna()

# Separate features (X) and target variable (y)
X = df[['STA-2']]  # Features
y = df['Sustainability']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the logistic regression model
logistic_model = LogisticRegression()

# Fit the model to the training data
logistic_model.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred = logistic_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print a classification report for more detailed metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7209302325581395
Classification Report:
               precision    recall  f1-score   support

  sustainable       0.69      0.92      0.79        24
unsustainable       0.82      0.47      0.60        19

     accuracy                           0.72        43
    macro avg       0.75      0.70      0.69        43
 weighted avg       0.75      0.72      0.70        43

